In [4]:
from sklearn.datasets import fetch_openml
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Load the NIST dataset (you might need to adjust the version)
nist = fetch_openml('mnist_784', version=1, parser='auto')
X = np.array(nist.data.astype('float64'))
y = nist.target.astype('int')

In [5]:
# Labeling: +1 for 4, -1 for 9, 0 for unknown
y_binary = np.where(y == 4, 1, np.where(y == 9, -1, 0))

# Use a smaller subset of the data for testing
subset_size = 1000  # You can adjust this number based on your available memory
subset_indices = np.random.choice(len(X), size=subset_size, replace=False)
X_subset = X[subset_indices]
y_binary_subset = y_binary[subset_indices]

In [6]:
# Step 1: Construct the K-NN Graph for Subset
k = 6  # Number of nearest neighbors
nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(X_subset)
distances, indices = nbrs.kneighbors(X_subset)
A = np.zeros((len(X_subset), len(X_subset)))

# Create the adjacency matrix based on k-NN for Subset
for i in range(len(X_subset)):
    for j in indices[i]:
        A[i, j] = 1
        A[j, i] = 1  # Assuming an undirected graph

In [7]:
# Step 2: Define the Known Labels Signal and Diagonal Matrix C for Subset
class_1_label = 1  # +1 for class 1 (number 4)
class_2_label = -1  # -1 for class 2 (number 9)

# Define known labels signal s_known_subset
s_known_subset = np.zeros(len(X_subset))
s_known_subset[y_binary_subset == class_1_label] = 1
s_known_subset[y_binary_subset == class_2_label] = -1

# Create the diagonal matrix C based on s_known_subset
C_subset = np.diag(s_known_subset)

In [8]:
# Step 3: Compute precomputable matrices M1 and M2 for Subset
I_subset = np.identity(len(A))
M1_subset = np.dot((I_subset - A).T, (I_subset - A))
M2_subset = np.dot(C_subset.T, C_subset)

In [9]:
# Step 4: Find the Optimal Alpha for Subset
alpha_values = np.linspace(0.01, 1.0, 100)  # Adjust the range and number of points as needed
best_alpha_subset = None
min_error_subset = np.inf

for alpha in alpha_values:
    s_predicted_subset = 2 * alpha * np.linalg.inv(M1_subset + 2 * alpha * M2_subset).dot(M2_subset).dot(s_known_subset)
    error_subset = 0.5 * np.linalg.norm(s_predicted_subset - np.dot(A, s_predicted_subset)) ** 2 + alpha * np.linalg.norm(np.dot(C_subset, s_predicted_subset) - np.dot(C_subset, s_known_subset)) ** 2
    if error_subset < min_error_subset:
        min_error_subset = error_subset
        best_alpha_subset = alpha

In [10]:
# Step 5: Compute the Final Predicted Signal for Subset
s_predicted_subset = 2 * best_alpha_subset * np.linalg.inv(M1_subset + 2 * best_alpha_subset * M2_subset).dot(M2_subset).dot(s_known_subset)

In [12]:
# Classify s(predicted_subset) as -1 if between 0 and -1, else +1
s_predicted_subset_classified = np.where(s_predicted_subset < 0, -1, 1)

In [13]:
# Print s(known) and s(predicted) for Subset
print("s(known) for Subset:")
print(s_known_subset)

print("s(predicted) for Subset:")
print(s_predicted_subset_classified)

s(known) for Subset:
[ 0.  0.  0.  0.  0. -1.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  1.
  0.  0.  0.  0.  0.  0.  0.  0. -1.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0. -1.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0. -1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.
  0.  0.  0.  0.  0.  1.  0. -1. -1. -1.  0.  0.  1.  0.  0. -1.  0.  0.
  0. -1.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0. -1.  1.  0.  0.  1.  0.  0.  0.  1.  1.  0.
  0.  0. -1.  0.  0.  0.  0. -1.  1.  0. -1.  0.  0.  0. -1.  0.  0.  0.
  1.  0. -1.  0.  1.  1.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 -1.  0.  0.  1.  0.  0.  0.  0. -1.  0. -1.  0.  0.  0.  0.  0.  0. -1.
  0.  0.  0. -1.  0.  0. -1.  